In [37]:
from garmin_fit_sdk import Decoder, Stream

stream = Stream.from_file('../data/13887398069.fit')
decoder = Decoder(stream)
messages, errors = decoder.read()

def printSkip():
    print("===========================================================")

def is_number(string):
    try:
        float(string)
        return True
    except ValueError:
        return False
'''
keys = messages.keys()
for key in keys:
    if not is_number(key) and key not in ["event_mesgs", "record_mesgs"]:
        print("#" + key + ":")
        print(messages[key])
        printSkip()
'''
print(len(messages["record_mesgs"]))
for i in range(50):
    print(messages["record_mesgs"][i])    
    printSkip()

#file_id_mesgs: general info about Garmin device and activity
#file_creator_mesgs: software version
#event_mesgs: general info about event? (ex. alert for Garmin live track?)
#device_info_mesgs: info of all devices and sensors owned
#device_settings_mesgs: time and specific device settings
#user_profile_mesgs: user profile info and settings
#sport_mesgs: activity sport
#zones_target_mesgs: HR and Power tresholds

#record_mesgs: all activity info!!!

#lap_mesgs: lap data
#time_in_zone_mesgs: HR and Power zones spent during the activity
#session_mesgs: training session info recap
#activity_mesgs: activity brief recap


12150
{'timestamp': datetime.datetime(2024, 12, 1, 8, 50, 21, tzinfo=datetime.timezone.utc), 'position_lat': 545875074, 'position_long': 111538078, 'distance': 4.87, 'accumulated_power': 125, 'altitude': 327.4, 'speed': 4.749, 'power': 125, 61: 4111, 66: 1793, 'heart_rate': 112, 'cadence': 66, 'temperature': 12, 'fractional_cadence': 0.0, 'left_pco': 0, 'right_pco': 0, 'enhanced_speed': 4.749, 'enhanced_altitude': 327.4}
{'timestamp': datetime.datetime(2024, 12, 1, 8, 50, 22, tzinfo=datetime.timezone.utc), 'position_lat': 545874797, 'position_long': 111538766, 'distance': 10.04, 'accumulated_power': 159, 'altitude': 322.0, 'speed': 4.899, 'power': 34, 61: 4110, 66: 1793, 'heart_rate': 113, 'cadence': 74, 'temperature': 12, 'fractional_cadence': 0.0, 'left_pco': 0, 'right_pco': 0, 'left_power_phase': [1.4062500219726566, 70.31250109863284], 'left_power_phase_peak': [22.500000351562505, 45.00000070312501], 'enhanced_speed': 4.899, 'enhanced_altitude': 322}
{'timestamp': datetime.datetime